## 필터링 로직 추가

- 새로운 데이터의 상품코드가 기 학습된 인코더 안에 있는지 확인
    - 없을 경우 마더코드 검색
    - 동일 마더코드 안에서 동일 상품군 검색
    - 동일 상품군 안에서 단가 비교를 통한 대체 데이터 반환
- 일자와 날씨 등의 다른 데이터는 그대로 넣어준다

In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
data = pd.read_csv("reference/encoded_data.csv")
data.head()

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,date,week,...,g_2,g_3,s_0,s_1,w_0,w_1,w_2,w_3,w_4,w_5
0,2019-01-01 06:00:00,20.0,296,891,테이트 남성 셀린니트3종,6,39900,2099000.0,2019-01-01 00:00:00,1,...,1,0,0,0,0,0,0,0,0,0
1,2019-01-01 06:00:00,20.0,296,898,테이트 여성 셀린니트3종,6,39900,4371000.0,2019-01-01 00:00:00,1,...,1,0,0,0,0,0,0,0,0,0
2,2019-01-01 06:20:00,20.0,296,891,테이트 남성 셀린니트3종,6,39900,3262000.0,2019-01-01 00:00:00,1,...,1,0,1,0,0,0,0,0,0,0
3,2019-01-01 06:20:00,20.0,296,898,테이트 여성 셀린니트3종,6,39900,6955000.0,2019-01-01 00:00:00,1,...,1,0,1,0,0,0,0,0,0,0
4,2019-01-01 06:40:00,20.0,296,891,테이트 남성 셀린니트3종,6,39900,6672000.0,2019-01-01 00:00:00,1,...,1,0,0,1,0,0,0,0,0,0


In [18]:
train = data[['마더코드', '상품코드', '상품군', '판매단가']]
train.head(2)

,마더코드,상품코드,상품군,판매단가
0,296,891,6,39900
1,296,898,6,39900


In [4]:
m_classes = np.load('reference/마더코드_classes.npy')
p_classes = np.load('reference/상품코드_classes.npy')
s_classes = np.load('reference/stage_classes.npy')
g_classes = np.load('reference/상품군_classes.npy')

In [6]:
p_classes

array(['200000', '200001', '200002', ..., '202510', '202512', '202513'],
      dtype=object)

In [9]:
test = pd.read_excel('test.xlsx', header = 1)
test.head()

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액
0,2020-06-01 06:20:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN
1,2020-06-01 06:40:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN
2,2020-06-01 07:00:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN
3,2020-06-01 07:20:00,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,NaN
4,2020-06-01 07:40:00,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,NaN


In [10]:
sample = test[['마더코드', '상품코드', '상품군', '판매단가']]
sample.head(2)

,마더코드,상품코드,상품군,판매단가
0,100650,201971,의류,59800
1,100650,201971,의류,59800


In [37]:
m_classes

array(['100000', '100001', '100002', '100006', '100009', '100010',
       '100013', '100015', '100016', '100017', '100018', '100019',
       '100020', '100021', '100022', '100023', '100024', '100025',
       '100026', '100027', '100028', '100029', '100031', '100032',
       '100033', '100034', '100036', '100037', '100038', '100039',
       '100040', '100041', '100042', '100043', '100044', '100045',
       '100046', '100047', '100048', '100049', '100050', '100051',
       '100052', '100053', '100054', '100055', '100056', '100057',
       '100058', '100061', '100062', '100063', '100064', '100065',
       '100066', '100067', '100069', '100070', '100074', '100075',
       '100076', '100078', '100079', '100080', '100081', '100083',
       '100084', '100085', '100086', '100087', '100088', '100089',
       '100090', '100091', '100093', '100094', '100095', '100096',
       '100097', '100098', '100099', '100100', '100101', '100102',
       '100103', '100104', '100107', '100109', '100111', '1001

In [35]:
np.where(g_classes == '의류')[0][0]

6

In [56]:
train.loc[train['상품군'] == np.where(g_classes == '의류')[0][0]]

,마더코드,상품코드,상품군,판매단가
0,296,891,6,39900
1,296,898,6,39900
2,296,891,6,39900
3,296,898,6,39900
4,296,891,6,39900
...,...,...,...,...
35305,228,727,6,299000
35306,228,727,6,299000
35337,304,921,6,79000
35338,304,921,6,79000


In [51]:
sample.iloc[0].values

array([100650, 201971, '의류', 59800], dtype=object)

In [58]:
final = []
values = sample.values
for i in range(len(values)):
    # 상품코드가 인코더 안에 있는지 확인
    if values[i][1] in p_classes:
        final.append(values[i])
    else:
        # 마더코드가 인코더 안에 있는지 확인
        if values[i][0] in m_classes:
            temp = train.loc[train['마더코드'] == np.where(m_classes == values[i][1])[0][0]]
            # 판매단가 차가 최소인 row를 추가함.
            temp['sub'] = abs(temp['판매단가'] - values[i][3])
            final.append(train.iloc[temp['sub'].idxmin()].values)
        else:
            if values[i][2] in g_classes:
                temp = train.loc[train['상품군'] == np.where(g_classes == values[i][2])[0][0]]
                # 판매단가 차가 최소인 row를 추가함.
                temp['sub'] = abs(temp['판매단가'] - values[i][3])
                final.append(train.iloc[temp['sub'].idxmin()].values)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

...

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [61]:
print(final[:5])

[array([  403,  1190,     6, 59800]), array([  403,  1190,     6, 59800]), array([  403,  1190,     6, 59800]), array([  534,  1600,     5, 69900]), array([  534,  1600,     5, 69900])]


In [62]:
print(values[:5])

[[100650 201971 '의류' 59800]
 [100650 201971 '의류' 59800]
 [100650 201971 '의류' 59800]
 [100445 202278 '속옷' 69900]
 [100445 202278 '속옷' 69900]]


## RandomForest with label encoded(no binary code)

In [63]:
data.columns

Index(['방송일시', '노출(분)', '마더코드', '상품코드', '상품명', '상품군', '판매단가', '취급액', 'date',
       'week', 'time', 'hour', 'prime', 'real_date', 'IsHoliday', '지속휴일수',
       'TEMP', 'HUM', 'weekofyear', 'stage', 'm_0', 'm_1', 'm_2', 'm_3', 'm_4',
       'm_5', 'm_6', 'm_7', 'm_8', 'm_9', 'p_0', 'p_1', 'p_2', 'p_3', 'p_4',
       'p_5', 'p_6', 'p_7', 'p_8', 'p_9', 'p_10', 'g_0', 'g_1', 'g_2', 'g_3',
       's_0', 's_1', 'w_0', 'w_1', 'w_2', 'w_3', 'w_4', 'w_5'],
      dtype='object')

In [90]:
X = data[['노출(분)', '마더코드', '상품코드', '상품군', '판매단가', 'week',
         'hour', 'prime', 'IsHoliday', '지속휴일수', 'TEMP', 
          'HUM', 'weekofyear', 'stage']]
Y = data['취급액']
display(X.head(2))
print("X shape :", X.shape)
print("Y shape :", Y.shape)

,노출(분),마더코드,상품코드,상품군,판매단가,week,hour,prime,IsHoliday,지속휴일수,TEMP,HUM,weekofyear,stage
0,20.0,296,891,6,39900,1,6,0,1.0,1.0,-6.576974,63.524958,0,0
1,20.0,296,898,6,39900,1,6,0,1.0,1.0,-6.576974,63.524958,0,0


X shape : (35379, 14)
Y shape : (35379,)


In [66]:
from sklearn.ensemble import RandomForestRegressor

In [67]:
rf = RandomForestRegressor(max_depth = 10,
                          random_state = 0)
rf.fit(X, Y)

RandomForestRegressor(max_depth=10, random_state=0)

In [101]:
from sklearn.metrics import accuracy_score
from sklearn.utils import check_array
def mean_absolute_percentage_error(y_true, y_pred): 
#     y_true, y_pred = check_array(y_true, y_pred)

    ## Note: does not handle mix 1d representation
    #if _is_1d(y_true): 
    #    y_true, y_pred = _check_1d_array(y_true, y_pred)

    return ('MAPE', np.mean(np.abs((y_true - y_pred) / y_true)))

In [69]:
rf.score(X, Y)

0.6922708397590764

In [94]:
Y = Y[np.newaxis, :]
Y.shape

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  """Entry point for launching an IPython kernel.


(1, 35379)

In [102]:
y_pred = rf.predict(X)
y_pred = y_pred.reshape(1, -1)
mean_absolute_percentage_error(Y, y_pred)

('MAPE', 0.8015259074675415)

In [86]:
type(y_pred)

numpy.ndarray

In [99]:
y_pred

array([[ 7743331.17332456,  7743331.17332456, 12392200.74571698, ...,
        26637026.12076516, 25191015.83450143, 24768765.62076518]])

In [87]:
type(Y)

numpy.ndarray

In [100]:
Y

array([[2.09900e+06, 4.37100e+06, 3.26200e+06, ..., 1.04392e+08,
        1.37650e+07, 4.66080e+07]])

In [103]:
check_array(Y)

array([[2.09900e+06, 4.37100e+06, 3.26200e+06, ..., 1.04392e+08,
        1.37650e+07, 4.66080e+07]])